In [1]:
import numpy as np

import lspi
# import ipdb
# ipdb.set_trace()

# Experiment 2 : LSPI on the *Inverted-Pendulum* environment
---------- 
First, we create a function that takes as input a policy and computes the average number of balancing steps.

In [2]:
def score(agent):
    _, episode_lengths = lspi.utils.evaluate_policy(agent,
                                                    agent.env,
                                                    max_length=3000,
                                                    n_eval_episodes=10)
    return int(np.mean(episode_lengths))

We perform *LSPI* with RBF features : $9$ means spanning the grid $\{-\pi/4,0,+\pi/4\}\times\{-1,0,+1\}$ and variance $\sigma=1$.

## Experiment 2.1 : *Inverted-Pendulum* environment for $1000$ episodes 
For the first experiment, the agent collect samples by playing for $1000$ episodes.

In [3]:
# build the environment
nA = 3
env = lspi.envs.SimulatorEnv(np.eye(nA))

# build the agent
grids = [[-1., 0., 1.], [-1., 0., 1.], [-1., 0., 1.]]
centers = lspi.agents.RadialAgent.get_centers_from_grids(grids)
sigma = 1.

agent = lspi.agents.RadialAgent(env, centers, sigma)


/opt/homebrew/Caskroom/miniconda/base/envs/py39/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [4]:

# build the trainer
gamma = 0.95
memory_size = 1000
memory_type = 'episode'
eval_type = 'batch'
baseline = lspi.baselines.LSPolicyIteration(env, agent, gamma, memory_size,
                                            memory_type, eval_type)

# build the memory
baseline.init_memory()
print('memory size = {}'.format(len(baseline.memory)))

# run the algorithm
n_iter = 10
steps = score(agent)
print('iteration = {:02d} - average number of balancing steps : {:04d}'.format(
    0, steps))
for it in range(1, n_iter + 1):
    baseline.train_step()
    steps = score(agent)
    print('iteration = {:02d} - average number of balancing steps : {:04d}'.
          format(it, steps))

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

## Experiment 2.2 : Number of training episodes vs. Score
For the second experiment, we vary the number of training episodes.

In [ ]:
range_episodic_memory = np.linspace(50, 1000, 20).astype(int)
n_trials = 10
results = []
for memory_size in range_episodic_memory:
    results.append([])

    # build the trainer
    baseline = lspi.baselines.LSPolicyIteration(env, agent, gamma, memory_size,
                                                memory_type, eval_type)
    for _ in range(n_trials):

        # build the memory
        baseline.init_memory()
        # initialize the agent
        agent.init_weights()

        # run the algorithm
        for it in range(1, n_iter + 1):
            baseline.train_step()
        steps = score(agent)
        results[-1].append(steps)

    print('n_episodes = {:04d} - average number of balancing steps : {:04d}'.
          format(memory_size, int(np.mean(results[-1]))))

In [ ]:
import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.title('Inverted pendulum (LSPI): Average balancing steps')
plt.xlabel('Number of training episodes')
plt.ylabel('Steps')
plt.xlim(0, 1000)
plt.ylim(0, 3010)

x = range_episodic_memory

y_mean = np.mean(results, 1)
plt.plot(x, y_mean, color='blue')

y_max = np.max(results, 1)
plt.plot(x, y_max, '--', color='pink')

y_min = np.min(results, 1)
plt.plot(x, y_min, '--', color='pink')

y_std = np.std(results, 1)
plt.fill_between(x, y_mean - y_std, y_mean + y_std, color='blue', alpha=.1)

plt.show()